In [39]:
#特征选择已经结束，依据新的数据集进行验证与找出最优模型
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
% matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import gc
from collections import Counter
import copy
from sklearn.preprocessing import StandardScaler
from collections import Counter


def minmax_target(x_train, x_test, y_train):
    import category_encoders as ce
    encoder = ce.TargetEncoder(cols=discrete_list, drop_invariant=False).fit(x_train, y_train)
    x_train = encoder.transform(x_train)  # 基于训练集得到编码器
    x_test = encoder.transform(x_test)

    std = StandardScaler()
    X_train[continue_list] = std.fit_transform(X_train[continue_list])
    X_test[continue_list] = std.transform(X_test[continue_list])

    return x_train, x_test


def read_csv(file_name, num_rows):
    return pd.read_csv(file_name, nrows=num_rows)


def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df


train = reduce_mem_usage(pd.read_csv("D:/2021软件服务外包比赛/result/" + 'train_choose.csv'))
test = reduce_mem_usage(pd.read_csv("D:/2021软件服务外包比赛/result/" + 'test_choose.csv'))
X_train = train.drop(['emd_lable2'], axis=1)
X_test = test.drop(['emd_lable2'], axis=1)
y_train = train['emd_lable2'].to_numpy()
y_test = test['emd_lable2'].to_numpy()
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(Counter(y_train), Counter(y_test))  # 比较训练集与测试集的分布情况

feature_list = list(X_train.columns.array)
discrete_list = ['pax_name', 'pax_passport', 'pref_orig_y3_3', 'seg_dep_time']
continue_list = list(set(feature_list) - set(discrete_list))
X_train, X_test = minmax_target(X_train, X_test, y_train)

Memory usage after optimization is: 1.68 MB
Decreased by 48.9%
Memory usage after optimization is: 0.42 MB
Decreased by 48.9%
(18745, 22) (4687, 22) (18745,) (4687,)
Counter({0: 17565, 1: 1180}) Counter({0: 4392, 1: 295})


# 先试用随机森林对特征选择后的数据集进行分类

In [40]:
#随机森林的实验——》全都预测为0了好像，过拟合非常严重啊
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

dTree = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0, n_jobs=-1)
dTree.fit(X_train, y_train)
y_pred = dTree.predict(X_test)
from sklearn.metrics import balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test))
print("auc=", roc_auc_score(y_true=y_test, y_score=dTree.predict_proba(X_test)[:, 1]))

balanced_accuracy_score= 0.5
f1= 0.0
precision_score= 0.0
recall_score= 0.0
auc= 0.7732275169028433


# 随机森林——调参实验

In [41]:
#随机森林调参
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier



# model 
clf = RandomForestClassifier(n_jobs=-1)

# Set the parameters by cross-validation

tuned_parameters = {
                    'n_estimators': [50, 100, 200]
                    ,'criterion': ['gini', 'entropy']
                    ,'max_depth': [2, 5]
                    ,'max_features': ['log2', 'sqrt', 'int']
                    ,'bootstrap': [True, False]
                    ,'warm_start': [True, False]
                    }

scores = ['average_precision']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf, tuned_parameters, cv=5,
                       scoring='%s' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for average_precision

Best parameters set found on development set:

{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 200, 'warm_start': False}

Grid scores on development set:

0.595 (+/-0.108) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'n_estimators': 50, 'warm_start': True}
0.616 (+/-0.104) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'n_estimators': 50, 'warm_start': False}
0.611 (+/-0.131) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'n_estimators': 100, 'warm_start': True}
0.617 (+/-0.106) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'n_estimators': 100, 'warm_start': False}
0.619 (+/-0.089) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'n_estimators': 200, 'warm_start': True}
0.616 (+/-0.110) for {'bootst

# 采用SMOTE过采样技术，查看平衡数据集的随机森林调参效果

In [42]:
#过拟合非常严重，使用不平衡数据集的处理方法！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！


from imblearn.over_sampling import SMOTE

smo = SMOTE(random_state=0)
X_train, y_train = smo.fit_sample(X_train, y_train)
print(Counter(y_train))

Counter({0: 17565, 1: 17565})


In [44]:
#随机森林调参——》使用平衡数据集后效果有所改善！

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier



# model 
clf = RandomForestClassifier(n_jobs=-1)

# Set the parameters by cross-validation

tuned_parameters = {
                    'n_estimators': [20 , 50, 100, 200]
                    ,'criterion': ['gini', 'entropy']
                    ,'max_depth': [3,5,10]
                    ,'max_features': ['log2', 'sqrt', 'int']
                    ,'bootstrap': [True, False]
                    ,'warm_start': [True, False]
                    }

scores = ['average_precision']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf, tuned_parameters, cv=5,
                       scoring='%s' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()J48

# Tuning hyper-parameters for average_precision

Best parameters set found on development set:

{'bootstrap': True, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'log2', 'n_estimators': 100, 'warm_start': True}

Grid scores on development set:

0.963 (+/-0.028) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 20, 'warm_start': True}
0.963 (+/-0.025) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 20, 'warm_start': False}
0.966 (+/-0.026) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 50, 'warm_start': True}
0.965 (+/-0.023) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 50, 'warm_start': False}
0.967 (+/-0.025) for {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'log2', 'n_estimators': 100, 'warm_start': True}
0.970 (+/-0.022) for {'bootstrap':

              precision    recall  f1-score   support

           0       0.96      0.89      0.93      4392
           1       0.23      0.47      0.31       295

    accuracy                           0.86      4687
   macro avg       0.59      0.68      0.62      4687
weighted avg       0.92      0.86      0.89      4687




In [32]:
#调参出来的模型计算其具体的结果看一下：
clf = RandomForestClassifier(n_jobs=-1,bootstrap=True, criterion='gini',max_depth=10, max_features='log2',n_estimators=20, warm_start=True)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test))
print("auc=", roc_auc_score(y_true=y_test, y_score=dTree.predict_proba(X_test)[:, 1]))

balanced_accuracy_score= 0.6823017967954061
f1= 0.3050108932461874
precision_score= 0.2247191011235955
recall_score= 0.4745762711864407
auc= 0.7732275169028433


# 梯度提升决策树调参

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=0) 
tuned_parameters = {
                    'n_estimators': [10 , 50, 100, 200,400]
                    ,'learning_rate': [0.1,0.5,1,3,5]
                    ,'max_depth': [1,3,5,10]
                    }

scores = ['f1']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf, tuned_parameters, cv=5,
                       scoring='%s' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for roc_auc

Best parameters set found on development set:

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}

Grid scores on development set:

0.927 (+/-0.023) for {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 10}
0.958 (+/-0.025) for {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 50}
0.964 (+/-0.022) for {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 100}
0.968 (+/-0.020) for {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200}
0.969 (+/-0.019) for {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 400}
0.961 (+/-0.021) for {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 10}
0.968 (+/-0.019) for {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
0.970 (+/-0.017) for {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
0.970 (+/-0.016) for {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
0.970 (+/-0.015) for {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 400}
0.965 (+/-0.01

In [38]:
clf = GradientBoostingClassifier(random_state=0,n_estimators=200,learning_rate=0.1,max_depth=3) 
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("balanced_accuracy_score=", balanced_accuracy_score(y_pred=y_pred, y_true=y_test))
print("f1=", f1_score(y_pred=y_pred, y_true=y_test))
print("precision_score=", precision_score(y_pred=y_pred, y_true=y_test))
print("recall_score=", recall_score(y_pred=y_pred, y_true=y_test))
print("auc=", roc_auc_score(y_true=y_test, y_score=dTree.predict_proba(X_test)[:, 1]))

balanced_accuracy_score= 0.5727038374857214
f1= 0.22911694510739855
precision_score= 0.3870967741935484
recall_score= 0.16271186440677965
auc= 0.7732275169028433
